In [1]:
library(tidyverse)
library(brms)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()

Loading required package: Rcpp

Loading 'brms' package (version 2.17.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:


In [2]:
data_cues <- read_csv("../data/data_cues.txt")


Rows: 30 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (3): Glottocode, Language, Genus
dbl (4): Case_Marking, Tight_Semantics, Rigid_Order, Verb_Middle

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
trees <- ape::read.tree("../data/posterior.tree")
tree <- phangorn::maxCladeCred(trees)

In [4]:

tree_cor <- ape::vcv.phylo(tree, corr = TRUE)
tree_cor <- tree_cor[data_cues$Language, data_cues$Language]


In [5]:
## first we just do scaling

data_cues <- data_cues %>%
    mutate(
        x1 = scale(Case_Marking)
      , x2 = scale(Tight_Semantics)
      , x3 = scale(Rigid_Order)
      , x4 = scale(Verb_Middle))

In [6]:
## the baseline correlations

round(cor(data_cues[,8:11]), 2)

,x1,x2,x3,x4
x1,1.00,0.35,-0.77,-0.19
x2,0.35,1.00,-0.18,-0.37
x3,-0.77,-0.18,1.00,0.18
x4,-0.19,-0.37,0.18,1.00


In [7]:

rescale <- function(x, e = 0.0001) {
    y = ifelse(x == 1, x - e, x)
    ifelse(y == 0, y + e, y)
}


In [8]:
data_cues_3 <- data_cues %>%
    mutate(
        x1 = rescale(Case_Marking)
      , x2 = rescale(Tight_Semantics)
      , x3 = rescale(Rigid_Order)
      , x4 = rescale(Verb_Middle))


In [9]:
bm_2b <- brm(bf(mvbind(x1,x2,x3,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_2b

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 13 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 13 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta, beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity
         mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.52      0.37     0.87     2.32 1.00
sd(x2_Intercept)                   0.25      0.14     0.02     0.55 1.00
sd(x3_Intercept)                   1.54      0.26     1.01     2.04 1.00
sd(x4_Intercept)                   2.80      0.42     1.94     3.62 1.00
cor(x1_Intercept,

In [10]:
bm_3_1 <- brm(bf(mvbind(x1,x2) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_1

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 1 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.30      0.40     0.54     2.13 1.00
sd(x2_Intercept)                   0.31      0.15     0.03     0.59 1.01
cor(x1_Intercept,x2_Intercept)     0.46      0.34    -0.37     0.95 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   2487     4352
sd(x2_Intercept)                   1287     3373
cor(x1_Intercept,x2_Intercept)     5504     5338

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [11]:
bm_3_2 <- brm(bf(mvbind(x1,x3) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_2

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 109 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 109 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.44      0.36     0.83     2.24 1.00
sd(x3_Intercept)                   1.53      0.28     0.98     2.09 1.00
cor(x1_Intercept,x3_Intercept)    -0.83      0.13    -0.99    -0.52 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   2414     3029
sd(x3_Intercept)                   1494     3179
cor(x1_Intercept,x3_Intercept)     2027     3658

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [12]:
bm_3_3 <- brm(bf(mvbind(x1,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_3

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 44 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 44 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.34      0.41     0.54     2.19 1.00
sd(x4_Intercept)                   2.74      0.47     1.74     3.60 1.00
cor(x1_Intercept,x4_Intercept)    -0.44      0.19    -0.79    -0.03 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   1898     2625
sd(x4_Intercept)                   1094      919
cor(x1_Intercept,x4_Intercept)     2096     2056

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [13]:
bm_3_4 <- brm(bf(mvbind(x2,x3) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_4

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 99 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 99 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x2_Intercept)                   0.26      0.16     0.01     0.59 1.01
sd(x3_Intercept)                   1.53      0.32     0.88     2.13 1.00
cor(x2_Intercept,x3_Intercept)    -0.26      0.38    -0.92     0.64 1.00
                               Bulk_ESS Tail_ESS
sd(x2_Intercept)                    889     2357
sd(x3_Intercept)                   1155     2287
cor(x2_Intercept,x3_Intercept)      825     1276

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [14]:
bm_3_5 <- brm(bf(mvbind(x2,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_5

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 321 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 321 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x2_Intercept)                   0.28      0.15     0.02     0.58 1.01
sd(x4_Intercept)                   2.65      0.53     1.31     3.55 1.01
cor(x2_Intercept,x4_Intercept)    -0.43      0.31    -0.96     0.26 1.01
                               Bulk_ESS Tail_ESS
sd(x2_Intercept)                    655     1001
sd(x4_Intercept)                    603      588
cor(x2_Intercept,x4_Intercept)      569      423

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [15]:
bm_3_6 <- brm(bf(mvbind(x3,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_6

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 1 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 3 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 1 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x3_Intercept)                   1.59      0.28     1.02     2.15 1.00
sd(x4_Intercept)                   2.79      0.44     1.90     3.61 1.00
cor(x3_Intercept,x4_Intercept)     0.37      0.16     0.04     0.64 1.00
                               Bulk_ESS Tail_ESS
sd(x3_Intercept)                   1074     2217
sd(x4_Intercept)                   1411     1749
cor(x3_Intercept,x4_Intercept)     3224     5407

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_

In [16]:
bm_4_1 <- brm(bf(mvbind(x1,x3) ~ x4
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = b, resp = x1) # priors for beta
                   , prior(normal(0, 1), class = b, resp = x3) # priors for beta                     
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_1

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 116 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 116 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ x4 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ x4 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.37      0.35     0.77     2.14 1.00
sd(x3_Intercept)                   1.48      0.27     0.97     2.06 1.01
cor(x1_Intercept,x3_Intercept)    -0.79      0.15    -0.99    -0.44 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   2132     2639
sd(x3_Intercept)                   1161      825
cor(x1_Intercept,x3_Intercept)     2206     4104

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [17]:
bm_4_2 <- brm(bf(mvbind(x1,x4) ~ x3
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = b, resp = x1) # priors for beta
                   , prior(normal(0, 1), class = b, resp = x4) # priors for beta                                          
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_2

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 33 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 33 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ x3 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ x3 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   0.90      0.44     0.10     1.76 1.01
sd(x4_Intercept)                   2.69      0.44     1.72     3.52 1.00
cor(x1_Intercept,x4_Intercept)    -0.40      0.27    -0.88     0.23 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   1174     2300
sd(x4_Intercept)                   1376     1277
cor(x1_Intercept,x4_Intercept)     1268     1052

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [18]:
bm_4_3 <- brm(bf(mvbind(x3,x4) ~ x1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = b, resp = x3) # priors for beta
                   , prior(normal(0, 1), class = b, resp = x4) # priors for beta                                          
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_3

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 6 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 6 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x3 ~ x1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ x1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
  Draws: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup draws = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x3_Intercept)                   1.27      0.28     0.66     1.81 1.00
sd(x4_Intercept)                   2.70      0.45     1.77     3.55 1.00
cor(x3_Intercept,x4_Intercept)     0.29      0.18    -0.08     0.61 1.00
                               Bulk_ESS Tail_ESS
sd(x3_Intercept)                   1191     1124
sd(x4_Intercept)                   1101     1016
cor(x3_Intercept,x4_Intercept)     2686     3200

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul